In [1]:
import numpy as np
import os
import glob
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import collections
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_federated as tff
import random

In [2]:
data_dir_parent = "C:/Users/temmuz/Desktop/2022-2/proje/stanford/dataset"
client_train_dataset = collections.OrderedDict()
client_test_dataset = collections.OrderedDict()
ratio = 0.2
ratio0and5 = 0.2
for client_num in range(6):
    print(client_num)
    data_dir = data_dir_parent+"/"+str(client_num)
    train_image_names = []
    test_image_names = []
    labels_train = []
    labels_test = []
    for i in range(11):
        data_dir2 = data_dir+"/"+str(i)+"/*.jpg"
        if len(glob.glob(data_dir2))>0:
            temp_names=glob.glob(data_dir2)
            train, test = train_test_split(temp_names, test_size=0.2, random_state=42)
            test_image_names.extend(test)
            labels_test.extend(np.ones(len(test),dtype=int)*i)
            if i == 0 or i == 5:
                full_len = len(train)
                used_len = int(ratio0and5*full_len)
                used_names = train[:used_len]
                train_image_names.extend(used_names)
                labels_train.extend(np.ones(len(used_names),dtype=int)*i)
            else:
                train_image_names.extend(train)
                labels_train.extend(np.ones(len(train),dtype=int)*i)
    data_train = np.zeros((len(train_image_names),112,112,3))
    data_test = np.zeros((len(test_image_names),112,112,3))
    for i, name in enumerate(train_image_names):
        img1 = PIL.Image.open(name)
        data_train[i] = tf.keras.preprocessing.image.img_to_array(img1)/255
    for i, name in enumerate(test_image_names):
        img1 = PIL.Image.open(name)
        data_test[i] = tf.keras.preprocessing.image.img_to_array(img1)/255
    data_train = collections.OrderedDict((('label', labels_train), ('image', data_train)))
    data_test = collections.OrderedDict((('label', labels_test), ('image', data_test)))
    client_train_dataset[client_num] = data_train
    client_test_dataset[client_num] = data_test
train_dataset = tff.simulation.FromTensorSlicesClientData(client_train_dataset)
test_dataset = tff.simulation.FromTensorSlicesClientData(client_test_dataset)

0
1
2
3
4
5


In [3]:
def preprocess(dataset):

    def batch_format_fn(element):
        return collections.OrderedDict(
                x=element['image'],
                y=tf.reshape(element['label'], [-1, 1]))

    return dataset.shuffle(10000, seed=42).batch(128).map(batch_format_fn)

In [4]:
train_set = [preprocess(train_dataset.create_tf_dataset_for_client(x)) for x in train_dataset.client_ids]
test_set = [preprocess(test_dataset.create_tf_dataset_for_client(x)) for x in test_dataset.client_ids]